<a href="https://colab.research.google.com/github/dmachlanski/ncrm-causality-2021/blob/main/labs/day_2/Exercise_JOBS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Use the JOBS dataset. Here we evaluate against ATT and policy risk (R_pol).

In [ ]:
def calc_stats(y, t, e, te_pred):
    att = np.mean(y[t > 0]) - np.mean(y[(1 - t + e) > 1])

    att_pred = np.mean(te_pred[(t + e) > 1])
    bias_att = att_pred - att

    policy_value = policy_val(te_pred[e > 0])

    eps_att = np.abs(bias_att)
    r_pol = 1 - policy_value
    return eps_att, r_pol
	
def policy_val(y, t, e, te_pred):
    t_e = t[e > 0]
    y_e = y[e > 0]

    if np.any(np.isnan(te_pred)):
        return np.nan, np.nan

    policy = te_pred > 0.0
    treat_overlap = (policy == t_e) * (t_e > 0)
    control_overlap = (policy == t_e) * (t_e < 1)

    if np.sum(treat_overlap) == 0:
        treat_value = 0
    else:
        treat_value = np.mean(y_e[treat_overlap])

    if np.sum(control_overlap) == 0:
        control_value = 0
    else:
        control_value = np.mean(y_e[control_overlap])

    pit = np.mean(policy)
    policy_value = pit * treat_value + (1 - pit) * control_value

    return policy_value